# 1.Tests

In [13]:
"""Some envs and imports for testing"""

from stormvogel.stormpy_utils.model_checking import model_checking
from prob_minigrid2storm import convert_to_probabilistic_storm, load_env_configs, process_config
from stormvogel.result import Result
from stormvogel.extensions.visual_algos import policy_iteration


configs = load_env_configs()
#contains all the information about the environments that is needed for making a probabilistic env wrapper and converting to storm model.
crossing_env_info= process_config(configs[1])
crossing_env_instance= crossing_env_info['env_class'](**crossing_env_info['env_params'])
crossing_env_storm, env_mapping = convert_to_probabilistic_storm(crossing_env_instance, crossing_env_info['used_actions'], crossing_env_info['prob_distribution'])

distshift_env_info= process_config(configs[2])
distshift_env_instance= distshift_env_info['env_class'](**distshift_env_info['env_params'])
distshift_env_storm, distshift_env_mapping = convert_to_probabilistic_storm(distshift_env_instance, distshift_env_info['used_actions'], distshift_env_info['prob_distribution'])



### 1.1 Test model_checking

In [9]:
def test_model_checking(env_storm, env_name): 
    print("=== Model Info ===")
    print(f"Number of states: {len(env_storm.states)}")
    print(f"Available labels: {env_storm.get_labels()}")

    # Example safety properties
    lava_eventually_min = "Pmin=? [F \"lava\"]"    # Probability of eventually reaching lava
    # lava_eventually_max = "Pmax=? [F \"lava\"]"    # Probability of eventually reaching lava
    lava_never_min = "Pmin=? [G !\"lava\"]"        # Probability of never reaching lava
    # lava_never_max = "Pmax=? [G !\"lava\"]"        # Probability of never reaching lava
    goal_eventually_min= "Pmin=? [F \"goal\"]"    # Probability of eventually reaching goal
    # goal_eventually_max= "Pmax=? [F \"goal\"]"    # Probability of eventually reaching goal

    """ The code below can only be run in a jupyter notebook environment"""
    print("\n=== Safety Properties ===")
    print(f"Testing property: {lava_eventually_min}")
    result:Result = model_checking(env_storm, lava_eventually_min)
    print(result.values)
    # print(result.model.states)
    print(f"times of 1 in results: {[val for _, val in result.values.items() if val == 1]}")
    # print(f"\nTesting property: {lava_eventually_max}")
    # result:Result = model_checking(env_storm, lava_eventually_max) 
    # print(result.maximum_result())

    print(f"\nTesting property: {lava_never_min}")
    result:Result = model_checking(env_storm, lava_never_min)
    print(result.values)
    print(f"times of 1 in results: {[val for _, val in result.values.items() if val == 1]}")
    # print(f"\nTesting property: {lava_never_max}")
    # result:Result = model_checking(env_storm, lava_never_max)
    # print(result.maximum_result())

    print(f"\nTesting property: {goal_eventually_min}")
    result:Result = model_checking(env_storm, goal_eventually_min)
    print(result.values)
    print(f"times of 1 in results: {[val for _, val in result.values.items() if val == 1]}")
    # print(f"\nTesting property: {goal_eventually_max}")
    # result:Result = model_checking(env_storm, goal_eventually_max)
    # print(result.values)

test_model_checking(crossing_env_storm, "CrossingEnv")


=== Model Info ===
Number of states: 186
Available labels: {'None', 'goal', 'lava', 'init'}

=== Safety Properties ===
Testing property: Pmin=? [F "lava"]
WARN  (IterativeMinMaxLinearEquationSolver.cpp:191): Expected VI operator to be initialized for scheduler extraction. Initializing now, but this is inefficient.
WARN  (IterativeMinMaxLinearEquationSolver.cpp:191): Expected VI operator to be initialized for scheduler extraction. Initializing now, but this is inefficient.
{0: 0.03867471506908121, 1: 0.03882105481774512, 2: 0.038660259101353664, 3: 0.038658229596464165, 4: 0.04008900676525611, 5: 0.03868038457790509, 6: 0.03868014460428625, 7: 0.03865845304879231, 8: 0.03865614114701988, 9: 0.05119324183236812, 10: 0.038855483038796754, 11: 0.038855167847674524, 12: 0.03866280377253984, 13: 0.03867797303293159, 14: 0.03865405111535924, 15: 0.03867262115481645, 16: 0.038656381392508474, 17: 0.14843577562949095, 18: 0.04038928319111903, 19: 0.0403884199378839, 20: 0.03870129618890923, 21:

### 1.2 Test policy iteration

In [15]:

def test_policy_iteration(): 
    # Define what we want to optimize for
    goal_maximization = "P=?[F \"goal\"]"  # Maximize probability of reaching goal
    
    print(f"Running policy iteration to optimize: {goal_maximization}")
    print(f"This will find the policy that maximizes the chance of reaching the goal")
    
    result = policy_iteration(distshift_env_storm, prop=goal_maximization, visualize=True, clear = False)
    
    # You could also try different objectives:
    # safety_maximization = "Pmax=? [G !\"lava\"]"  # Maximize probability of never hitting lava
    # lava_minimization = "Pmin=? [F \"lava\"]"     # Minimize probability of hitting lava

test_policy_iteration()

Running policy iteration to optimize: P=?[F "goal"]
This will find the policy that maximizes the chance of reaching the goal


Output()

Output()

AttributeError: 'JSVisualization' object has no attribute 'clear'